In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd 
import dataload
import datetime as datetime
pd.set_option("max_rows", 500)
pd.set_option("max_columns", 500)
pd.set_option('float_format', '{:f}'.format)

import os 
import datetime
from tqdm.notebook import tqdm 
from functools import reduce
%matplotlib inline
tqdm.pandas()

# #1. Data preprocessing - 혜린

# 1. 데이터 로드

## 1-1. 내부 데이터

## 1-2. 기상청 데이터

In [ ]:
def load_file(weather) : 
    
    '''
    weather : 추출하고 싶은 날씨 데이터 변수 
    '''
    
    # folder path 설정 
    file_path = os.path.join(os.path.abspath(".."), 'data')
    file_list = os.listdir(file_path)
    
    # startswith('i') : i로 시작하는 파일 모두 불러오기 / endswith('csv') : csv 파일 모두 불러오기 
    data_file = sorted([file for file in file_list if file.startswith(weather)])
 
    # csv 파일들을 DataFrame으로 불러와서 concat
    df = pd.DataFrame()
    for d in data_file:
        data = pd.read_csv(os.path.join(file_path, d))
        df = pd.concat([df, data], axis=0).reset_index(drop=True)
    
    # index column 삭제 ..ㅎㅎ 
    df = df.drop('Unnamed: 0', axis=1)
    
    return df

### 1-2-1. 습도

In [ ]:
tqdm.pandas()

humid = load_file('humid')
humid['date'] = humid['tm'].progress_apply(lambda x : pd.to_datetime(x[:10]))
humid2 = humid.groupby(['date', 'aws_id']).max().reset_index()[['date', 'aws_id', 'hm_max']]

In [ ]:
# 지역명 부여 함수
def make_region(aws_id):
    region = {'서울':108, '부산':159, '대구':143, '인천':112, '광주':156, '대전':133, '울산':152, '수원':119, 
              '강릉':105, '청주':131, '천안':232, '전주':146, '여수':168, '안동':136, '창원':155, '제주':184}
    res = None
    for r, code in region.items():
        if aws_id == code:
             res = r
    return res

humid2['region'] = humid2['aws_id'].apply(make_region)

In [ ]:
# 해당 지역, 해당 년월의 평균치로 결측치를 대체하는 함수

def missing_value(data, aws_id, year, month, var):
    data['year'] = data['date'].apply(lambda x : x.year)
    data['month'] = data['date'].apply(lambda x : x.month)
    sample = data[(data.aws_id==aws_id)&(data.year==year)&(data.month==month)]
    alt = sample[var].mean() # 평균
    # 대체
    data[(data.aws_id==aws_id)&(data.year==year)&(data.month==month)] = data[(data.aws_id==aws_id)&(data.year==year)&(data.month==month)].fillna(alt) 
    data = data.drop(['year', 'month'], axis=1) # 월 변수 제거
    return data

humid2 = missing_value(humid2, 232, 2019, 3, 'hm_max') # 충남(천안)
humid2 = missing_value(humid2, 232, 2019, 4, 'hm_max') # 대구
humid2 = missing_value(humid2, 143, 2019, 4, 'hm_max') # 대구

### 1-2-2. 일조시간

In [ ]:
tqdm.pandas()

sun = load_file('sun')
sun['date'] = sun['tm'].progress_apply(lambda x : pd.to_datetime(x[:10]))
sun2 = sun.groupby(['date', 'stn_id']).max().reset_index()[['date', 'stn_id', 'sum_ss_hr']]
sun2.columns = ['date', 'aws_id', 'sum_ss_hr']

# 지역명 부여
sun2['region'] = sun2['aws_id'].apply(make_region)

### 1-2-3. 기압

In [ ]:
tqdm.pandas()

press = load_file('press')
press['date'] = press['tma'].progress_apply(lambda x : pd.to_datetime(x[:10]))
press2 = press.groupby(['date', 'stn_id']).max().reset_index()[['date', 'stn_id', 'max_pa']]
press2.columns = ['date', 'aws_id', 'max_pa']

# 지역명 부여
press2['region'] = press2['aws_id'].apply(make_region)

# 결측치 처리
press2 = missing_value(press2, 143, 2019, 4, 'max_pa') # 대구
press2 = missing_value(press2, 105, 2019, 12, 'max_pa') # 강릉
press2 = missing_value(press2, 105, 2020, 1, 'max_pa') # 강릉

### 1-2-4. 기온

In [ ]:
tqdm.pandas()

temp = load_file('temp')
temp['date'] = temp[#'tm'#].progress_apply(lambda x : pd.to_datetime(x[:10]))
    
    
# 평균기온, 최대기온, 최고기온
temp2 = temp.groupby(['date', 'aws_id']).max().reset_index()[['date', 'aws_id', 'hm_max']]

### 1-2-5. 강수

In [ ]:
tqdm.pandas()

rain = load_file('rain')
temp['date'] = temp[#'tm'#].progress_apply(lambda x : pd.to_datetime(x[:10]))
    
    
# 평균기온, 최대기온, 최고기온
temp2 = temp.groupby(['date', 'aws_id']).max().reset_index()[['date', 'aws_id', 'hm_max']]

### 1-2-6. 풍속

In [ ]:
tqdm.pandas()

wind = load_file('wind')
wind['date'] = wind[#'tm'#].progress_apply(lambda x : pd.to_datetime(x[:10]))
    
# 평균기온, 최대기온, 최고기온
temp2 = temp.groupby(['date', 'aws_id']).max().reset_index()[['date', 'aws_id', 'hm_max']]

## 1-3. 외부 데이터

### 1-3-1. 미세먼지

In [ ]:
def load_dust(year) : 
    
    # folder path 설정 
    file_path = os.path.join(os.path.abspath(".."), 'data\\' + year)
    file_list = os.listdir(file_path)
    
    # 파일 불러오기
    data_file = sorted([file for file in file_list if file.startswith(year)])
    
    # csv 파일들을 DataFrame으로 불러와서 concat
    df = pd.DataFrame()
    for d in data_file:
        data = pd.read_excel(os.path.join(file_path, d), engine='openpyxl')
        df = pd.concat([df, data], axis=0).reset_index(drop=True)
    df = df[~df.지역.isna()] # 지역값이 없는 경우 제거
    df = df[~df.PM10.isna()]
    df = df[~df.PM25.isna()]
    
    # 지역 리스트
    region = {'서울':108, '부산':159, '대구':143, '인천':112, '광주':156, '대전':133, '울산':152, '수원':119, 
              '강릉':105, '청주':131, '천안':232, '전주':146, '여수':168, '안동':136, '창원':155, '제주':184}
    
    # 시간별 데이터를 일별 데이터로 변환
    # 지역별 평균값
    data = pd.DataFrame()
    for r, code in region.items(): # r:지명, code:지역코드
        sample = df.copy()
        sample['date'] = sample['측정일시'].apply(lambda x : str(x)[:8])
        sample['region'] = sample['지역'].apply(lambda x : r if r in x else None)
        sample = sample[sample.region == r]
        sample['aws_id'] = code
        sample = sample.groupby(['date', 'region', 'aws_id']).mean().reset_index()[['date', 'region', 'aws_id', 'PM10', 'PM25']]
        data = pd.concat([data, sample], axis=0).reset_index(drop=True)

    return data

In [ ]:
dust2018 = load_dust('2018')
dust2019 = load_dust('2019')
dust2020 = load_dust('2020')

In [ ]:
dust = pd.concat([dust2018, dust2019, dust2020], axis=0).reset_index(drop=True)
dust.columns = ['date', 'region', 'aws_id', 'PM10', 'PM25']
dust['date'] = dust['date'].progress_apply(lambda x : pd.to_datetime(x))

# 2. 데이터 병합

In [ ]:
weather = reduce(lambda left, right : pd.merge(left, right, how='left', on=['date', 'aws_id', 'region']), [temp2, rain2, wind2, humid2, press2, dust, sun2])
# weather = weather[['date', 'aws_id', 'region', 'PM10', 'PM25', 'hm_max', 'sum_ss_hr', 'max_pa']]
weather['year'] = weather['date'].apply(lambda x : x.year)
weather1819 = weather[(weather.year==2018)|(weather.year==2019)].drop('year', axis=1)

# 3. 파생변수 생성

## 3-1. 날씨 관련 파생변수

In [ ]:
# 체감온도 : 외부에 있는 사람이나 동물이 바람과 한기에 노출된 피부로 부터 열을 빼앗길 때 느끼는 추운 정도를 나타내는 지수
weather['체감온도'] = 13.12 + 0.6215*weather['평균기온'] - 11.37 * weather['평균풍속'] * 0.16 + 0.3965 * weather['평균풍속'] * 0.16 * weather['평균기온']

# 열지수 : 기온과 습도에 따라 사람이 실제로 느끼는 더위를 지수화한 것
H = weather['1시간최대습도']; T = weather['평균기온']
RH = H / sum(H) * 100 # percentage
weather['열지수'] = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH

# 폭염여부 : 일 최고기온이 33℃ 이상인 날
weather['폭염여부'] = weather['최고기온'].apply(lambda x : 1 if x>=33 else 0)

# 강수여부 : 일강수량이 0.1mm 이상인 날
weather['강수여부'] = weather['일별강수량'].apply(lambda x : 1 if x>=0.1 else 0)

## 3-2. 시간 관련 파생변수

In [ ]:
weather['날짜'] = weather['날짜'].apply(lambda x : pd.to_datetime(x))
weather['연']  = weather['날짜'].dt.year
weather['월']  = weather['날짜'].dt.month
weather['일']  = weather['날짜'].dt.day
weather["분기"] = weather['날짜'].dt.quarter
weather['요일']  = weather['날짜'].dt.weekday # 월 0 화 1 수 2 목 3 금 4 토 5 일 6 

In [ ]:
# 공휴일 정보 추출 함수
def getHoliday(year):
    
    '''
    공공데이터포털 공휴일 정보 OPEN API  
    '''
    
    url = f'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear={year}&ServiceKey={key}&_type=json&numOfRows=20'
    response = requests.get(url)
    holidays = response.json()['response']['body']['items']['item']
    holidays = pd.DataFrame(holidays)
    holidays['locdate'] = holidays['locdate'].astype(str).apply(lambda x : '-'.join([x[:4], x[4:6], x[6:]]))
    
    return holidays

key = '8H1yac%2Bb0yetY2Waad%2BQIRU43O17onRUd7iR0k2p6%2B4i0yNYn3ym0cXgt3ZqWrR9uFw%2BDu%2B5quHbsXxjpsw5ng%3D%3D'

# 공휴일 관련 변수
holidays2018 = getHoliday(2018)
holidays2019 = getHoliday(2019)

holidays = pd.concat([holidays2018, holidays2019], axis=0, ignore_index = True)
holidays["locdate"] = holidays['locdate'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))
holidays = holidays.drop(['dateKind', 'seq'], axis=1)
holidays.columns = ['공휴일명', '공휴일여부', '날짜']

weather = weather.merge(holidays[['날짜', '공휴일명', '공휴일여부']], on='날짜', how='left')
weather['공휴일여부'] = weather['공휴일여부'].apply(lambda x : 1 if x == 'Y' else 0)

In [ ]:
# 주말 : [5,6]
weather['주말여부'] = weather['요일'].apply(lambda x : 1 if x in [5,6] else 0)

# 계절 : 봄 0 여름 1 가을 2 겨울 3 
weather["계절"] = weather["월"].apply(lambda x : 0 if x in [3,4,5] else 1 if x in [6,7,8] 
                                   else 2 if x in [9,10,11] else 3)

### 3-3. 번외) 결측치 처리

In [ ]:
# 해당 지역, 해당 년월의 평균치로 결측치를 대체하는 함수

def missing_value2(data, aws_id, year, var):
    month_list = data[(data[var].isnull())&(data.지점번호==aws_id)&(data.year==year)]['month'].unique()
    for month in month_list:
        sample = data[(data.지점번호==aws_id)&(data.year==year)&(data.month==month)&(~data[var].isnull())]
        alt = sample[var].mean() # 평균
        # 대체
        if np.isnan(alt)==True: # 해당 년,월의 변수값이 하나도 없으면 전년도 월평균으로 대체
            data.loc[(data.지점번호==aws_id)&(data.year==year)&(data.month==month),var] = data.loc[(data.지점번호==aws_id)&(data.month==month), var].mean()
        else:
            data.loc[(data.지점번호==aws_id)&(data.year==year)&(data.month==month),var] = alt 
    return data

### 3-3-1. `PM10`

In [ ]:
# 안동
weather = missing_value2(weather, 136, 2018, 'PM10')
weather = missing_value2(weather, 136, 2019, 'PM10')
# 제주
weather = missing_value2(weather, 184, 2018, 'PM10')
weather = missing_value2(weather, 184, 2019, 'PM10')
# 천안
weather = missing_value2(weather, 232, 2018, 'PM10')
weather = missing_value2(weather, 232, 2019, 'PM10')
# 강릉 
weather = missing_value2(weather, 105, 2018, 'PM10')
weather = missing_value2(weather, 105, 2019, 'PM10')

### 3-3-2. `PM25`

In [ ]:
# 안동
weather = missing_value2(weather, 136, 2018, 'PM25')
weather = missing_value2(weather, 136, 2019, 'PM25')
# 제주
weather = missing_value2(weather, 184, 2018, 'PM25')
weather = missing_value2(weather, 184, 2019, 'PM25')
# 천안
weather = missing_value2(weather, 232, 2018, 'PM25')
weather = missing_value2(weather, 232, 2019, 'PM25')
# 강릉 
weather = missing_value2(weather, 105, 2018, 'PM25')
weather = missing_value2(weather, 105, 2019, 'PM25')

### 3-3-3. 일조시간

In [ ]:
# 대구
weather = missing_value2(weather, 143, 2018, '일조시간합')
weather = missing_value2(weather, 143, 2019, '일조시간합')
# 부산 
weather = missing_value2(weather, 159, 2018, '일조시간합')
weather = missing_value2(weather, 159, 2019, '일조시간합')
# 서울
weather = missing_value2(weather, 108, 2018, '일조시간합')
weather = missing_value2(weather, 108, 2019, '일조시간합')

In [ ]:
# ======= output : weather_final2 =======

======= 여기 final_0606 만드는 코드 들어가야함